In [5]:
pip install PyPDF2

In [10]:
import PyPDF2

def read_pdf(file_path):
    try:
        with open(file_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            pdf_text = ""
            for page_num, page in enumerate(reader.pages):
                pdf_text += f"\n--- Page {page_num + 1} ---\n"
                pdf_text += page.extract_text()
            return pdf_text
    except FileNotFoundError:
        return "Error: File not found. Please check the file path."
    except Exception as e:
        return f"An error occurred: {e}"

def chat_with_pdf(content):
    print("Chat with your PDF! Type 'exit' to quit.")
    while True:
        user_query = input("You: ")
        print(" ")
        if user_query.lower() == 'exit':
            print("Chat ended. Goodbye!")
            break
        else:
            if user_query.lower() in content.lower():
                start_idx = content.lower().find(user_query.lower())
                snippet = content[start_idx:start_idx + 500]
                print(f"PDF Bot: Here's what I found: \n{snippet}...\n")
            else:
                print("PDF Bot: Sorry, I couldn't find anything related to your query in the document.")

file_path = "maths.pdf"
pdf_content = read_pdf(file_path)

if "Error" not in pdf_content:
    chat_with_pdf(pdf_content)
else:
    print(pdf_content)


Chat with your PDF! Type 'exit' to quit.
You: probability
 
PDF Bot: Here's what I found: 
PROBABILITY
2024-25

--- Page 2 ---
290 MATHEMA TICS
14.1.1   Occurrence of an event   Consider the experiment of throwing a die. Let E
denotes the event “ a number  less than 4 appears”. If actually ‘1’ had appeared on the
die then we say that event E has occurred. As a matter of fact if outcomes are 2 or 3,
we say that event E has occurred
Thus, the event E of a sample space S is said to have occurred if the outcome
ωof the experiment is such that ω∈ E. If the outcome ω is such that ω ∉ E, we ...

You: Range
 
PDF Bot: Here's what I found: 
range is the interval [0,1]
satisfying the following axioms
(i)For any event E,  P  (E) ≥0 (ii) P (S) = 1
(iii) If E and F are mutually exclusive events, then P(E ∪ F) = P(E) + P(F).
It follows from (iii) that P( φ) = 0. To prove this, we take F = φ and note that E and φ
are disjoint events. Therefore, from axiom (iii), we get
P (E ∪ φ) = P (E) + P ( φ)or P(